In [1]:
import pandas as pd
import os 
path_to_lafolette_exp1 = '/Users/thomasnegrello/Desktop/autoneuro/datasets/lafolette_2023/'
df = pd.read_csv(path_to_lafolette_exp1+'study1_processed.csv')
#last trial for each particiapent is the number 105 and is a preattentioncheck
condition = (df['trial'] == 105)
#condition = (df['trial_type'] == 'preattentioncheck') & (df['trial_type'].shift(-1) == 'initialization')
df_filtered = df[~condition]


In [2]:
# Remove practice
id_col = df_filtered['ID'][df_filtered['trial_type']!='preattentioncheck']

state_col = [1] * len(id_col)

reward_col = df_filtered['working_phone'].copy()
reward_col.dropna(inplace=True)

value_col = df_filtered['reported_q'].copy()
value_col.dropna(inplace=True)

In [3]:
# Create new DataFrame using the generated columns
fdf = pd.DataFrame({
    'id': id_col.reset_index(drop=True),
    's': state_col,
    'v': value_col.reset_index(drop=True),
    'r': reward_col.reset_index(drop=True)
})

In [4]:
fdf

,id,s,v,r
0,1,1,0.50,0.0
1,1,1,1.00,1.0
2,1,1,1.00,1.0
3,1,1,1.00,1.0
4,1,1,0.90,0.0
...,...,...,...,...
45495,455,1,0.17,0.0
45496,455,1,0.14,1.0
45497,455,1,0.81,0.0
45498,455,1,0.65,0.0


In [5]:
nb_subjects = int(fdf['id'].iloc[-1])
path_to_processed_lafolette_exp1 = '/Users/thomasnegrello/Desktop/autoneuro/datasets/datasets_processed/dataset_lafolette_exp1/'

# Save the required columns to their respective files
column_to_filename = {
    's': 'observations.csv',
    'v': 'values.csv',
    'r': 'rewards.csv',
}

In [6]:
for id in range(1, nb_subjects+1):
    subject_id =  int(id)
    subject = fdf[fdf['id'] == subject_id].reset_index()
    # Create the directory for the subject
    subject_folder = os.path.join(path_to_processed_lafolette_exp1, 'Sub' + str(subject_id))
    os.mkdir(subject_folder)

    for col, filename in column_to_filename.items():
        file_path = os.path.join(subject_folder, filename)
        subject[col].to_csv(file_path, header=False, index=False)
